In [1]:
import pandas as pd  
import numpy as np  
import os  
import sys  
import collections
from itertools import chain
import json
from IPython.display import display, HTML
import csv
import glob
import h5py
import re


In [2]:
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_colwidth', 0)
pd.options.display.float_format = lambda x: '{:,.2f}'.format(x) if x > 1 else '{:,.2%}'.format(x)

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [4]:
top_metrics_file = None
dict_features ={}
top_metrics_file = 'top_metrics_for_tbl.csv'
dict_features = {row[0] : row[1] for _, row in pd.read_csv(top_metrics_file).iterrows()}
df_features = pd.read_csv(top_metrics_file)
list_metrics = list(set(df_features['metric'] ))


In [5]:
# Get input H5 file
input_h5_file = []
input_h5_file = 'input_for_html_report.h5'


In [30]:
# Get base_file_name
with open('base_file_name.txt', 'r') as f:
    input_base_file_name = f.readline().rstrip()

In [6]:
# get Keys within the H5 file
f = h5py.File(input_h5_file, 'r')
list_h5_df = []
list_keys = list(f.keys())
hist_list_keys =  [i for i in list_keys if i.startswith('histogram_')]
tbl_list_keys = pd.DataFrame(list(set(list_keys) - set(hist_list_keys)))
tbl_list_keys.columns = ['metric']
del(list_keys)

# create table merging top required metrics to display and input provided
# ....................
tbl_top_values = pd.merge(df_features,tbl_list_keys,on='metric',how='inner')

In [37]:
to_print = pd.DataFrame(data = {'key' : 'Sample name', 'value' : input_base_file_name}, index = [0])
to_print

,key,value
0,Sample name,baseline_L3511_016796_HG001_rep1


In [7]:

HTML("<b>"+""+"</b>")
HTML("<hr/>")
HTML("<h2 style=\"font-size:20px;\">"+"Summary View: Main Metrics"+"</h2>")
HTML("<hr/>")
HTML("<b>"+""+"</b>")



In [8]:
separator = '___'
i_value= np.nan 
to_print = pd.DataFrame()

tbl_top = pd.DataFrame()
for temp_metric in tbl_top_values['metric'].unique():
    list_top_tbl = sub_top_tbl = pd.DataFrame()
    sub_top_tbl =  tbl_top_values[tbl_top_values['metric'] == temp_metric]
    df_h5_tbl = pd.read_hdf(input_h5_file, temp_metric).T
    df_h5_tbl = df_h5_tbl.reset_index()
    
    # stats_coverage is a multiindex dataframe    
    if (temp_metric.startswith('stats') ):
        df_h5_tbl['metric'] = df_h5_tbl['level_0'] + separator + df_h5_tbl['level_1']
        df_h5_tbl = df_h5_tbl.drop(columns=['level_0','level_1']).copy()
        df_h5_tbl.columns = ['value','key']
        df_h5_tbl = df_h5_tbl[['key', 'value']]        
        list_top_tbl = pd.merge(df_h5_tbl, sub_top_tbl ,on='key',how='inner')
        to_print = to_print.append(list_top_tbl)
        
    else:
        df_h5_tbl.columns = ['key','value']        
        list_top_tbl = pd.merge(df_h5_tbl, sub_top_tbl ,on='key',how='inner')
        to_print = to_print.append(list_top_tbl)
if type(to_print.loc[to_print.key == 'PCT_PF_READS', 'value'].values[0]) is int:
    to_print.loc[to_print.key == 'PCT_PF_READS', 'value'] = '100%'
to_print.reset_index().drop(columns = 'index')

    



,key,value,metric
0,TOTAL_READS,614332000,AlignmentSummaryMetrics
1,PCT_PF_READS,99.77%,AlignmentSummaryMetrics
2,PCT_PF_READS_ALIGNED,99.13%,AlignmentSummaryMetrics
3,PF_MISMATCH_RATE,20.15%,AlignmentSummaryMetrics
4,PF_INDEL_RATE,0.76%,AlignmentSummaryMetrics
5,MEAN_READ_LENGTH,224.68,AlignmentSummaryMetrics
6,MEAN_ALIGNED_READ_LENGTH,220.49,AlignmentSummaryMetrics
7,PF_BASES,137717547905,QualityYieldMetricsFlow
8,PF_Q30_BASES,91026891229,QualityYieldMetricsFlow
9,GC_NC_0_19,52.85%,GcBiasSummaryMetrics


In [9]:
HTML("<b>"+""+"</b>")
HTML("<hr/>")
HTML("<h2 style=\"font-size:20px;\">"+"Detailed View: All Metrics"+"</h2>")
HTML("<hr/>")
HTML("<b>"+""+"</b>")


In [10]:
to_print = pd.DataFrame()
sorted_keys  = tbl_list_keys['metric'].sort_values()

for tbl_key in sorted_keys:
    
    HTML("<br>"+"<br>"+"<b>"+"Metric type: "+tbl_key+"</b>"+"<br>")
    df_h5_tbl = pd.read_hdf(input_h5_file, tbl_key).T
    df_h5_tbl = df_h5_tbl.reset_index()
    

    if (tbl_key.startswith('stats') ):
        df_h5_tbl['metric'] = df_h5_tbl['level_0'] + separator + df_h5_tbl['level_1']
        df_h5_tbl = df_h5_tbl.drop(columns=['level_0','level_1']).copy()
        df_h5_tbl.columns = ['value','key']
        to_print = df_h5_tbl[['key', 'value']]        
        to_print

    else:
        to_print = df_h5_tbl
        to_print.columns = ['key','value'] 
        to_print
        


,key,value
0,CATEGORY,UNPAIRED
1,TOTAL_READS,614332000
2,PF_READS,612938364
3,PCT_PF_READS,99.77%
4,PF_NOISE_READS,0
5,PF_READS_ALIGNED,607635638
6,PCT_PF_READS_ALIGNED,99.13%
7,PF_ALIGNED_BASES,134730406449
8,PF_HQ_ALIGNED_READS,541239186
9,PF_HQ_ALIGNED_BASES,120730619622


,key,value
0,LIBRARY,L3511
1,UNPAIRED_READS_EXAMINED,607981005
2,READ_PAIRS_EXAMINED,0
3,SECONDARY_OR_SUPPLEMENTARY_RDS,10355345
4,UNMAPPED_READS,6350995
5,UNPAIRED_READ_DUPLICATES,39965875
6,READ_PAIR_DUPLICATES,0
7,READ_PAIR_OPTICAL_DUPLICATES,0
8,PERCENT_DUPLICATION,6.57%
9,ESTIMATED_LIBRARY_SIZE,NaN


,key,value
0,ACCUMULATION_LEVEL,All Reads
1,READS_USED,ALL
2,GC,0
3,WINDOWS,136305
4,READ_STARTS,16041
5,MEAN_BASE_QUALITY,14
6,NORMALIZED_COVERAGE,58.04%
7,ERROR_BAR_WIDTH,0.46%
8,SAMPLE,NaN
9,LIBRARY,NaN


,key,value
0,ACCUMULATION_LEVEL,All Reads
1,READS_USED,ALL
2,WINDOW_SIZE,100
3,TOTAL_CLUSTERS,624687345
4,ALIGNED_READS,618336350
5,AT_DROPOUT,5.45
6,GC_DROPOUT,0.00%
7,GC_NC_0_19,52.85%
8,GC_NC_20_39,89.49%
9,GC_NC_40_59,1.08


,key,value
0,READ_LENGTH_AVG_Q_ABOVE_30,0
1,READ_LENGTH_AVG_Q_ABOVE_25,0
2,TOTAL_READS,614332000
3,PF_READS,612938364
4,READ_LENGTH,224
5,TOTAL_BASES,137807226292
6,PF_BASES,137717547905
7,Q20_BASES,119903460651
8,PF_Q20_BASES,119828940593
9,Q30_BASES,91082044488


,key,value
0,GENOME_TERRITORY,"2,937,690,299.00"
1,MEAN_COVERAGE,41.79
2,SD_COVERAGE,57.44
3,MEDIAN_COVERAGE,40.00
4,MAD_COVERAGE,8.00
5,PCT_EXC_ADAPTER,0.00%
6,PCT_EXC_MAPQ,0.00%
7,PCT_EXC_DUPE,5.87%
8,PCT_EXC_UNPAIRED,0.00%
9,PCT_EXC_BASEQ,0.14%


,key,value
0,GENOME_TERRITORY,2937690299
1,MEAN_COVERAGE,33.91
2,SD_COVERAGE,26.09
3,MEDIAN_COVERAGE,35
4,MAD_COVERAGE,8
5,PCT_EXC_ADAPTER,0.00%
6,PCT_EXC_MAPQ,8.63%
7,PCT_EXC_DUPE,4.81%
8,PCT_EXC_UNPAIRED,0
9,PCT_EXC_BASEQ,10.36%


,key,value
0,Genome___percentile_5,21.51
1,Genome___percentile_10,26.10
2,Genome___median_coverage,41.29
3,Genome___median_coverage_normalized,100.00%
4,Genome___percent_larger_than_05_of_genome_median,96.02
5,Genome___percent_larger_than_025_of_genome_median,98.99
6,Genome___percent_over_or_equal_to_10x,99.12
7,Genome___percent_over_or_equal_to_20x,96.56
8,Exome (WG)___percentile_5,25.92
9,Exome (WG)___percentile_10,31.14


In [11]:
# FIN #